## 기존 데이터셋을 학습에 적절하게 전처리

라이브러리 임포트 및 기본 설정

In [ ]:
import zipfile
import json
from pathlib import Path

# --- 설정값 (실제 환경에 맞게 파일명을 수정해야 합니다) ---
INPUT_ZIP_FILENAMES = [
    "TL_01.식당카페_01.입장_및_이용안내.zip",
    "TL_01.식당카페_02.자리안내.zip",
    "TL_01.식당카페_03.메뉴추천.zip",
    "TL_01.식당카페_04.메뉴주문.zip",
    "TL_01.식당카페_05.식음료서빙.zip",
    "TL_01.식당카페_06.결제_및_할인_포인트적립_안내.zip",
]
INPUT_DIR = Path("./") # ZIP 파일이 위치한 디렉토리
DEFAULT_CONTEXT = "카페"

# 모든 대화 쌍을 저장할 리스트 (파일로 저장하지 않고 메모리에 유지)
all_dialogue_pairs = []

print(f"총 {len(INPUT_ZIP_FILENAMES)}개의 압축 파일을 처리합니다.")

데이터 추출 및 가공 함수 정의

In [ ]:
def extract_dialogue_pairs(dialogue_data, context):
    """
    단일 대화 JSON 데이터(dialogue 리스트)에서 연속적인 발화 쌍(Q&A)을 추출합니다.
    (Turn N의 발화가 input_text, Turn N+1의 발화가 output_text가 됩니다.)
    """
    pairs = []
    
    if not isinstance(dialogue_data, list) or len(dialogue_data) < 2:
        return pairs

    for i in range(len(dialogue_data) - 1):
        input_turn = dialogue_data[i]
        output_turn = dialogue_data[i+1]
        
        input_text = input_turn.get('utterance', '').strip()
        output_text = output_turn.get('utterance', '').strip()
        
        if input_text and output_text:
            pairs.append({
                "context": context,
                "input_text": input_text,
                # 참고: train_data_cafe_500.json의 일반화/청크화 로직은 제외하고 
                #      원본 발화만 사용합니다.
                "output_text": output_text 
            })
            
    return pairs

모델 학습 클래스

In [ ]:
class CafeModelTrainer:
    """
    추출된 데이터를 받아 모델 학습을 시뮬레이션하는 클래스입니다.
    실제 학습 로직은 여기에 구현되어야 합니다 (예: 토크나이저, 데이터 로더, 모델 정의).
    """
    def __init__(self, model_name: str = "T5_Cafe_Model"):
        self.model_name = model_name
        self.data_size = 0
        print(f"🧠 모델 트레이너 초기화: {self.model_name}")

    def preprocess_data(self, raw_data_list: list) -> list:
        """
        raw_data_list를 모델 학습에 적합한 형태로 전처리합니다.
        (예: 문장 토큰화, 청크 분할, ID 매핑 등)
        """
        print(f"⚙️ 총 {len(raw_data_list)}개의 Q&A 쌍을 전처리 중...")
        
        # --- (실제 구현 필요 부분) ---
        # dataset_pretraining.ipynb에서 제시된 토큰화 및 청크화 로직을 여기에 적용해야 합니다.
        # 예: input_text, output_text를 분해하여 여러 개의 청크 학습 샘플 생성
        # 현재는 전처리 단계를 시뮬레이션합니다.
        
        processed_data = [{"features": f"Input: {d['input_text']}", "labels": f"Output: {d['output_text']}"} 
                          for d in raw_data_list]
        self.data_size = len(processed_data)
        print(f"✅ 전처리 완료. 총 {self.data_size}개의 학습 샘플 생성 완료.")
        return processed_data

    def train(self, raw_data_list: list):
        """
        인메모리 데이터를 사용하여 모델 학습을 시작합니다.
        """
        if not raw_data_list:
            print("❌ 학습할 데이터가 없습니다.")
            return

        # 1. 데이터 전처리
        processed_data = self.preprocess_data(raw_data_list)

        # 2. 모델 로드 및 학습 (시뮬레이션)
        print("🚀 모델 로드 및 학습 시작...")
        
        # --- (실제 구현 필요 부분) ---
        # torch.load(model_path) 또는 tf.keras.Model(...) 등을 사용하여 모델을 로드하고
        # data_loader, optimizer를 설정한 후 model.fit(...)을 실행해야 합니다.
        
        print(f"🎉 모델 '{self.model_name}' 학습 완료! (학습 샘플 수: {self.data_size})")

main 실행

In [ ]:
# 1. 데이터 추출
for zip_filename in INPUT_ZIP_FILENAMES:
    zip_path = INPUT_DIR / zip_filename
    print(f"\n--- {zip_filename} 처리 시작 ---")
    
    if not zip_path.exists():
        print(f"🚨 경고: {zip_filename} 파일을 찾을 수 없습니다. (스킵)")
        continue

    try:
        with zipfile.ZipFile(zip_path, 'r') as zf:
            json_files = [f for f in zf.namelist() if f.endswith('.json')]
            
            for json_file in json_files:
                try:
                    with zf.open(json_file, 'r') as f:
                        raw_data = f.read().decode('utf-8')
                        data = json.loads(raw_data)
                    
                    dialogue_list = data.get('dialogue', [])
                    pairs = extract_dialogue_pairs(dialogue_list, DEFAULT_CONTEXT)
                    # 파일에 저장하지 않고 바로 리스트에 추가
                    all_dialogue_pairs.extend(pairs)
                    
                except json.JSONDecodeError:
                    pass # 오류 파일은 무시
                except Exception:
                    pass
    except zipfile.BadZipFile:
        print(f"🚨 치명적 오류: {zip_filename}이 손상되었거나 유효한 ZIP 파일이 아닙니다.")
    except Exception as e:
        print(f"🚨 기타 오류 발생: {e}")

print(f"\n✅ 데이터 추출 완료. 총 {len(all_dialogue_pairs)}개의 원본 Q&A 쌍을 메모리에 로드했습니다.")

# 2. 모델 학습 연동 (파일 저장 없이 바로 진행)
trainer = CafeModelTrainer()
trainer.train(all_dialogue_pairs)

# 모델 성능 검증

In [ ]:
import zipfile
import json
from pathlib import Path
import random # 성능 검증 시뮬레이션을 위한 라이브러리
from typing import List, Dict

# --- 설정값 (실제 환경에 맞게 파일명을 수정해야 합니다) ---
VALIDATION_ZIP_FILENAMES = [
    "VL_01.식당카페_01.입장_및_이용안내_01.json.zip",
    "VL_01.식당카페_02.자리안내_01.json.zip",
    "VL_01.식당카페_03.메뉴추천_01.json.zip",
    "VL_01.식당카페_04.메뉴주문_01.json.zip",
    "VL_01.식당카페_05.식음료서빙_01.json.zip",
    "VL_01.식당카페_06.결제_및_할인_포인트적립_안내_01.json.zip",
]
INPUT_DIR = Path("./") # ZIP 파일이 위치한 디렉토리
DEFAULT_CONTEXT = "카페"

# 모든 검증 쌍을 저장할 리스트
all_validation_pairs = []

print(f"총 {len(VALIDATION_ZIP_FILENAMES)}개의 검증 압축 파일을 처리합니다.")

model validation class

In [ ]:
class CafeModelValidator:
    """
    훈련된 모델을 로드하고 검증 데이터를 이용해 성능을 측정합니다.
    """
    def __init__(self, model_path: str = "path/to/trained_cafe_model"):
        self.model_path = model_path
        self._load_model()
        print(f"✅ 모델 검증기 초기화 완료.")

    def _load_model(self):
        """
        훈련된 모델을 로드하는 과정을 시뮬레이션합니다. 
        실제 환경에서는 torch.load(), tf.keras.models.load_model() 등을 사용해야 합니다.
        """
        print(f"🧠 {self.model_path}에서 모델 로드 중...")
        # self.model = ActualModelClass.load(self.model_path)
        self.model_loaded = True 
        
    def _simulate_prediction(self, input_text: str) -> str:
        """
        모델이 input_text에 대해 응답을 생성하는 과정을 시뮬레이션합니다.
        """
        # 실제 모델 추론: model.generate(input_token_ids)
        
        # 시뮬레이션: 80% 확률로 정답과 유사한 응답, 20% 확률로 다른 응답 생성
        if random.random() < 0.8:
            return f"모델 응답: {input_text}에 대한 적절한 응답"
        else:
            return "모델 응답: 죄송합니다. 다시 말씀해 주시겠어요?"

    def calculate_metrics(self, data: List[Dict]):
        """
        모델의 응답과 정답(Ground Truth)을 비교하여 성능 지표를 계산합니다.
        """
        predictions = []
        references = []
        
        print(f"📊 총 {len(data)}개의 검증 샘플로 성능 측정 시작...")
        
        for item in data:
            input_text = item['input_text']
            # 모델이 생성한 응답
            predicted_output = self._simulate_prediction(input_text)
            # 정답
            ground_truth = item['output_text']
            
            predictions.append(predicted_output)
            references.append(ground_truth)
            
            # 실제 모델 학습에서는 토큰 ID 레벨에서 Loss, Perplexity 계산

        # --- (실제 구현 필요 부분) ---
        # 1. HuggingFace datasets나 torchmetrics 등을 사용하여 실제 메트릭 계산
        #    - BLEU, ROUGE, Exact Match, Perplexity 등
        
        # 시뮬레이션 결과
        simulated_loss = random.uniform(0.5, 1.5)
        simulated_accuracy = random.uniform(0.75, 0.95)
        simulated_bleu = random.uniform(0.60, 0.85)
        
        print("\n=== 모델 성능 검증 결과 ===")
        print(f"  - 검증 샘플 수: {len(data)}")
        print(f"  - 평균 검증 손실 (Loss): {simulated_loss:.4f} (Simulated)")
        print(f"  - 발화 일치 정확도 (Accuracy): {simulated_accuracy:.2%} (Simulated)")
        print(f"  - BLEU 점수 (자연어 생성 품질): {simulated_bleu:.4f} (Simulated)")
        print("============================\n")

# ---

검증 데이터 추출 및 평가

In [ ]:
# 1. 검증 데이터 추출
print("=== 1. 검증 데이터 추출 시작 ===")
for zip_filename in VALIDATION_ZIP_FILENAMES:
    zip_path = INPUT_DIR / zip_filename
    
    if not zip_path.exists():
        print(f"🚨 경고: {zip_filename} 파일을 찾을 수 없습니다. (스킵)")
        continue

    try:
        with zipfile.ZipFile(zip_path, 'r') as zf:
            json_files = [f for f in zf.namelist() if f.endswith('.json')]
            
            for json_file in json_files:
                try:
                    with zf.open(json_file, 'r') as f:
                        raw_data = f.read().decode('utf-8')
                        data = json.loads(raw_data)
                    
                    dialogue_list = data.get('dialogue', [])
                    pairs = extract_dialogue_pairs(dialogue_list, DEFAULT_CONTEXT)
                    all_validation_pairs.extend(pairs)
                    
                except json.JSONDecodeError:
                    continue
                except Exception:
                    continue
    except Exception as e:
        print(f"🚨 오류 발생 ({zip_filename}): {e}")

print(f"\n✅ 데이터 추출 완료. 총 {len(all_validation_pairs)}개의 검증 쌍을 메모리에 로드했습니다.")

# 2. 모델 성능 검증
if all_validation_pairs:
    validator = CafeModelValidator()
    validator.calculate_metrics(all_validation_pairs)
else:
    print("❌ 검증 데이터가 없으므로 성능 검증을 수행할 수 없습니다.")